In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [6]:
scales = [1]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.mode(x)

error = 0.0001
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/ops.py:141: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result = torch.tensor(mode_within(x_1d, error))
/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/ops.py:149: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give 

In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [1]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":1,"param_scale":1,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,1],"model_input_scales":[1],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 2.4716269969940186 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 39.5
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[8983262245034475985, 4321099455052339947, 15405541267776549565, 129283864358809647], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [7849486104498536031, 9471811025875477344, 14212922505027326017, 3214504566215810058]]], 'proof': '1b19ffa6c6c1acf8642c10c5c344f69ce2cd01685c120bf03d3de7e0b122681021c85e9b5584f01753ade3120b23542ef75ece395b0d26c42a642251702a26e82417dbcd205df1a96ed527c848db0804f3ac99ba9a384e5769a82d129a670bc52e5164457f536eb7b815f493bf35ee59516bb836aca287486526ee38eab1327d18d022a1ae1c64caf300d0ee523a229a2e565cb123b066fd98a393950acdb4e71f932306cd93f1dfe1ca4af0e35c62b19fd1dc9ec3a05cb4a4262e7f7a71228521f45ceca7b9f281d23683574c8988ac3860419351e3b8b384575de9fff058d6114feb0e7efa6092e3b5d9f0fbacd60b0e67a57efd41caede859f219a4aff96f2d5d10901a0a12f93baa454561035fc90003db423d67a7d6150ded18df0e28be13bf5204270f29acddb50fdf84a631b27ba058bb31f8657d0bb8c25beffabb4e1a728f3f8f6baae9f177561633629abf14d1b78f6f36128d4b17d30c76e90094

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 39.5
